<a href="https://colab.research.google.com/github/harshitEbPandey/transferLearningNER/blob/mlp_models/Telugu_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
!pip install fasttext

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import optimizers
from keras import regularizers
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import keras
# import fasttext
from tensorflow import random
from sklearn.metrics import f1_score, classification_report, ConfusionMatrixDisplay

### Tokenizing and saving txt (X_train_telu, Y_train_telu)

In [ ]:
from datasets import load_dataset
te_ner = load_dataset('ai4bharat/naamapadam', 'te')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam/te/1.0.0/c1b045180d60b208d2468bdad897d04461f08c7137c04a85220697b1bef7df9a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# analysing the hindi dataset
te_ds = te_ner.map()

# shapes of each
print(f"Train: {te_ds['train'].shape}")
print(f"Test: {te_ds['test'].shape}")
print(f"Validation: {te_ds['validation'].shape}")

# loading each
train_te_ds = te_ds['train'].to_pandas()
test_te_ds = te_ds['test'].to_pandas()
vali_te_ds = te_ds['validation'].to_pandas()

Train: (507741, 2)
Test: (861, 2)
Validation: (2700, 2)


In [ ]:
file_loc = '/content/drive/MyDrive/CSE 572/Datasets/Telugu'

In [ ]:
# combine each of the words in the individual sentences and create a common df
# text output is of the form in X_train.txt: `tokens`
# text output is of the form in Y_train.txt: `ner_tags`
import pandas as pd
with open(file_loc + '/X_train_te_ds.txt', 'w') as f1, open(file_loc + '/Y_train_te_ds.txt', 'w') as f2:
  for index, row in train_te_ds.iterrows():
      f1.write('\n'.join(row['tokens']) + '\n')
      f2.write('\n'.join(map(str, row['ner_tags'])) + '\n')

### Loading the tokens and tags from txt

In [ ]:
# all tokens and tags are presaved...
# taking them input and making unique dictionary
# Open the input file and read the words
with open(file_loc + '/X_train_te_ds.txt', 'r') as f:
    words = f.read().splitlines()

In [ ]:
with open(file_loc + '/Y_train_te_ds.txt', 'r') as f:
    tags = f.read().splitlines()

In [ ]:
len(tags) == len(words)

True

In [ ]:
word_tag_unique = {}
for idx, word in enumerate(words):
  if word not in word_tag_unique:
    word_tag_unique[word] = [tags[idx]]
  elif word in word_tag_unique and tags[idx] not in word_tag_unique[word]:
    word_tag_unique[word].append(tags[idx])

In [ ]:
with open(file_loc + '/train_te_unique_tokens.txt', 'w') as f:
  for word in word_tag_unique.keys():
    f.write(word + '\n')

### Training fasttext on unique words dimensions = 30

In [ ]:
# run fasttext model
model = fasttext.train_unsupervised(file_loc + '/train_te_unique_tokens.txt', model='skipgram', lr=0.05, dim=30, ws=5, epoch=5)

In [ ]:
# save the model
model.save_model(file_loc + '/fasttext_model_train_te_unique_tokens.bin')

In [ ]:
# load the model
model = fasttext.load_model(file_loc + '/fasttext_model_train_te_unique_tokens.bin')

In [ ]:
# every word should have an embedding now
# can get using model[word]

### Loading the training set

In [ ]:
# constructing x_train_df
word_dict = {i: word for i, word in enumerate(words)}
X_train_df = pd.DataFrame.from_dict(word_dict, orient='index', columns=['tokens'])

In [ ]:
# constructing y_train_df
tag_dict = {i: tag for i, tag in enumerate(tags)}
Y_train_df = pd.DataFrame.from_dict(tag_dict, orient='index', columns=['tags'])

### Saving word vectors in chunks

In [ ]:
def process_chunk(start_idx, end_idx, df):
    # Get the words for the chunk
    words = df.loc[start_idx:end_idx, 'tokens'].tolist()

   # Get the vectors for the words using the fastText model
    vectors = [model[word] for word in words]

    # list of index
    idx = range(start_idx, end_idx + 1)

    # Create a new dataframe with the vectors
    vectors_df = pd.DataFrame({'tokens': words, 'vectors': vectors}, index = idx)

    # # Save the chunk to a CSV file
    vectors_df.to_csv(file_loc + f'/vectors/te_lite_vectors_{start_idx}_{end_idx}.csv', index=False)

In [ ]:
X_train_df.shape

(5140539, 1)

In [ ]:
# Loop over the dataframe in chunks
chunk_size = 1000000
for i in range(0, len(X_train_df), chunk_size):
    start_idx = i
    end_idx = min(i + chunk_size, len(X_train_df)-1)
    process_chunk(start_idx, end_idx, X_train_df)
    # Print progress
    print(f'Saved vectors for words {start_idx} to {end_idx} to CSV file.')

Saved vectors for words 0 to 1000000 to CSV file.
Saved vectors for words 1000000 to 2000000 to CSV file.
Saved vectors for words 2000000 to 3000000 to CSV file.
Saved vectors for words 3000000 to 4000000 to CSV file.
Saved vectors for words 4000000 to 5000000 to CSV file.
Saved vectors for words 5000000 to 5140538 to CSV file.


In [ ]:
# encode the labels
label_encoder = LabelEncoder()
label_encoder.fit(Y_train_df.tags.values)
Y_train_te_enc = label_encoder.transform(Y_train_df.tags.values)

In [ ]:
# get each class' weights
unique, counts = np.unique(Y_train_te_enc, return_counts=True)
class_weights = dict(zip(unique, np.round(sum(counts) / counts)))

# more the number of records - lesser the weights are (for balancing dataset)
print(class_weights)

{0: 1.0, 1: 15.0, 2: 18.0, 3: 26.0, 4: 28.0, 5: 25.0, 6: 119.0}


In [ ]:
Y_train_te_enc.shape

(5140539,)

### Training MLP CLassifier

In [ ]:
np.unique(Y_train_te_enc)

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
num_features = 30
print(num_features)
num_classes = 7

30


In [ ]:
# Set the random seed
np.random.seed(42)
random.set_seed(42)

mlp = Sequential()
mlp.add(Dense(units=100, activation='relu', input_dim=num_features))
mlp.add(Dense(units=num_classes, activation='softmax'))
mlp.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Save Test Data

In [ ]:
with open(file_loc + '/X_test_te_ds.txt', 'w') as f1, open(file_loc + '/Y_test_te_ds.txt', 'w') as f2:
  for index, row in test_te_ds.iterrows():
      f1.write('\n'.join(row['tokens']) + '\n')
      f2.write('\n'.join(map(str, row['ner_tags'])) + '\n')

### Load test data

In [ ]:
# Load the pre-trained fastText model
model = fasttext.load_model(file_loc + '/fasttext_model_train_te_unique_tokens.bin')

# Open the input file and read the words
with open(file_loc + '/X_test_te_ds.txt', 'r') as f:
    words = [line.strip() for line in f]

# Create a dictionary with the words as values and integer indices as keys
word_dict = {i: word for i, word in enumerate(words)}

# Create a dataframe from the dictionary
df_test = pd.DataFrame.from_dict(word_dict, orient='index', columns=['tokens'])

# Loop over the dataframe in chunks
start_idx = 0
end_idx = len(df_test) - 1
process_chunk(start_idx, end_idx, df_test)
# Print progress
print(f'Saved test vectors for words {start_idx} to {end_idx} to CSV file.')

# get the labels in another dataframe
with open(file_loc +'/Y_test_te_ds.txt', 'r') as f:
    test_tags = [line.strip() for line in f if line.strip()]

tag_dict = {i: tag for i, tag in enumerate(test_tags)}

df_test_tags = pd.DataFrame.from_dict(tag_dict, orient='index', columns=['tags'])

X_test = pd.read_csv(file_loc + f'/vectors/te_lite_vectors_{start_idx}_{end_idx}.csv')
X_test.head()

Saved test vectors for words 0 to 8736 to CSV file.


,tokens,vectors
0,ఈ,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
1,ధర్మాసనంలో,[-1.7161961e-03 2.1603059e-03 -5.5950083e-04 ...
2,న్యాయమూర్తులు,[-0.001526 0.00181337 -0.00089099 0.000376...
3,జస్టిస్,[-1.3272591e-03 -7.6551054e-04 -6.1240268e-04 ...
4,ఎల్ఎన్,[ 1.1380144e-03 1.6251347e-03 -6.7872164e-04 ...


In [ ]:
# preparing data for prediction

target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']

y_test_te_ds = label_encoder.transform(df_test_tags.tags.values)

X_test = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_test.vectors])

Y_test = y_test_te_ds

y_true_labels = Y_test

In [ ]:
# training model pipeline (in chunks)

# Load data in chunks, and train
# training sets
# Loop over the dataframe in chunks
def run_model(epoch):
  chunk_size = 1000000
  for i in range(0, len(X_train_df), chunk_size):
      start_idx = i
      end_idx = min(i + chunk_size, len(X_train_df)-1)
      X_train = pd.read_csv(file_loc + f'/vectors/te_lite_vectors_{start_idx}_{end_idx}.csv')
      X_train = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_train.vectors])
      Y_train = Y_train_te_enc[start_idx: end_idx + 1]

      # Print progress
      print(f'Loaded vectors for words {start_idx} to {end_idx} to CSV file.')
      
      # train mlp on this
      mlp.fit(X_train, Y_train, class_weight=class_weights, epochs = epoch, batch_size = 200)

      # predict on test
      Y_test_pred = mlp.predict(X_test)

      # get pred labels
      y_pred_labels = np.argmax(Y_test_pred, axis=1)

      # print classification report
      print(classification_report(y_true_labels, y_pred_labels, target_names=target_names))

      # save model
      mlp.save(file_loc + f'/models/mlp_model_te_lite_{epoch}_epochs.h5')

      # Print progress
      print(f'Completed training with vectors for words {start_idx} to {end_idx}')

### TL - TL Baseline - 50 epochs
```
             precision    recall  f1-score   support

     Class 0       0.88      0.50      0.64      6528
     Class 1       0.22      0.42      0.29       607
     Class 2       0.27      0.42      0.33       530
     Class 3       0.18      0.37      0.24       263
     Class 4       0.14      0.48      0.22       253
     Class 5       0.23      0.59      0.33       482
     Class 6       0.08      0.50      0.14        74

    accuracy                           0.49      8737
   macro avg       0.29      0.47      0.31      8737
weighted avg       0.71      0.49      0.55      8737
```

Completed training with vectors for words 0 to 5140538

In [ ]:
# running for 100 epochs
run_model(100)

Loaded vectors for words 0 to 1000000 to CSV file.
Epoch 1/100
5001/5001 [==============================] - 16s 3ms/step - loss: 13.1392 - accuracy: 0.0543
Epoch 2/100
5001/5001 [==============================] - 15s 3ms/step - loss: 12.9571 - accuracy: 0.0678
Epoch 3/100
5001/5001 [==============================] - 14s 3ms/step - loss: 12.8318 - accuracy: 0.0738
Epoch 4/100
5001/5001 [==============================] - 13s 3ms/step - loss: 12.7478 - accuracy: 0.0805
Epoch 5/100
5001/5001 [==============================] - 14s 3ms/step - loss: 12.6775 - accuracy: 0.0854
Epoch 6/100
5001/5001 [==============================] - 16s 3ms/step - loss: 12.6107 - accuracy: 0.0989
Epoch 7/100
5001/5001 [==============================] - 15s 3ms/step - loss: 12.5451 - accuracy: 0.1170
Epoch 8/100
5001/5001 [==============================] - 14s 3ms/step - loss: 12.4742 - accuracy: 0.1763
Epoch 9/100
5001/5001 [==============================] - 14s 3ms/step - loss: 12.4063 - accuracy: 0.2484
Epoc

### Same as above but 100 epochs:
```
              precision    recall  f1-score   support

     Class 0       0.89      0.51      0.65      6528
     Class 1       0.28      0.32      0.30       607
     Class 2       0.29      0.43      0.35       530
     Class 3       0.16      0.38      0.23       263
     Class 4       0.15      0.49      0.22       253
     Class 5       0.23      0.58      0.33       482
     Class 6       0.04      0.45      0.08        74

    accuracy                           0.49      8737
   macro avg       0.29      0.45      0.31      8737
weighted avg       0.72      0.49      0.56      8737
```

In [ ]:
# get the labels in another dataframe
with open(file_loc +'/Y_test_te_ds.txt', 'r') as f:
    test_tags = [line.strip() for line in f if line.strip()]
tag_dict = {i: tag for i, tag in enumerate(test_tags)}

df_test_tags = pd.DataFrame.from_dict(tag_dict, orient='index', columns=['tags'])

# X_test = pd.read_csv(file_loc + f'/vectors/te_lite_vectors_{start_idx}_{end_idx}.csv')
# X_test.head()
df_test_tags.tags.values

array(['0', '0', '0', ..., '0', '0', '0'], dtype=object)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_test = pd.read_csv(file_loc + f'/vectors/te_lite_vectors_0_8736.csv')

# preparing data for prediction

target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']

label_encoder = LabelEncoder()
label_encoder.fit(df_test_tags.tags.values)
Y_test_te_ds = label_encoder.transform(df_test_tags.tags.values)

X_test = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_test.vectors])

Y_test = Y_test_te_ds

y_true_labels = Y_test

In [ ]:
mlp = keras.models.load_model(file_loc + '/models/mlp_model_te_lite_100_epochs.h5')

# predict on test
Y_test_pred = mlp.predict(X_test)

# get pred labels
y_pred_labels = np.argmax(Y_test_pred, axis=1)

# print classification report
print(classification_report(y_true_labels, y_pred_labels, target_names=target_names))

274/274 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

     Class 0       0.89      0.51      0.65      6528
     Class 1       0.28      0.32      0.30       607
     Class 2       0.29      0.43      0.35       530
     Class 3       0.16      0.38      0.23       263
     Class 4       0.15      0.49      0.22       253
     Class 5       0.23      0.58      0.33       482
     Class 6       0.04      0.45      0.08        74

    accuracy                           0.49      8737
   macro avg       0.29      0.45      0.31      8737
weighted avg       0.72      0.49      0.56      8737



In [ ]:
def eval_results_hmm(y_test, y_pred, clf_report=True, conf_matrix=False, exclude_0=True):
    """
        Report evaluation metrics for the NER model (HMM)
    """
    labels = np.arange(1,7) if exclude_0 else np.arange(0,7)
    print("Weighted F1 of HMM = {:.4f}".format(f1_score(y_test, y_pred, average='weighted', labels=labels)))
    if clf_report:
        print(classification_report(y_test, y_pred, labels=labels, digits=3, zero_division=0))
    if conf_matrix:
        ConfusionMatrixDisplay.from_predictions(y_test, y_pred, labels=labels)
        plt.show()
    print("================================================================\n")

In [ ]:
eval_results_hmm(y_true_labels, y_pred_labels)

Weighted F1 of HMM = 0.2922
              precision    recall  f1-score   support

           1      0.283     0.316     0.299       607
           2      0.290     0.432     0.347       530
           3      0.162     0.376     0.227       263
           4      0.146     0.494     0.225       253
           5      0.229     0.581     0.328       482
           6      0.042     0.446     0.077        74

   micro avg      0.194     0.434     0.268      2209
   macro avg      0.192     0.441     0.250      2209
weighted avg      0.234     0.434     0.292      2209


